In [27]:
import io
import os
from pathlib import Path
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient

In [18]:
def load_enviroment():
    # Enviroment variables loader that works both locally and in DataBricks.
    absolute_path = Path.absolute(Path(os.getcwd()))

    for parent in absolute_path.parents:
        if str(parent).endswith("spark-pipeline"):
            repo_path = parent
            break

    enviroment_variable_path = os.path.join(repo_path, "config", ".env")

    load_dotenv(enviroment_variable_path)

In [24]:
load_enviroment()

KAGGLE_FILE = os.environ["KAGGLE_FILE"]
BRONZE_CONTAINER = os.environ["BRONZE_CONTAINER"]
SILVER_CONTAINER = os.environ["SILVER_CONTAINER"]
AZURE_STORAGE_ACCOUNT_CONNECTION_STRING = os.environ["AZURE_STORAGE_ACCOUNT_CONNECTION_STRING"]

In [ ]:
blob_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_ACCOUNT_CONNECTION_STRING)

In [ ]:
spark = SparkSession.builder.appName("AzureBlobProcessing").getOrCreate()